In [0]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np



In [0]:
with open('anna.txt','r') as f:
  text=f.read()

In [0]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [0]:
char = tuple(set(text))
int2char=dict(enumerate(char))
char2int={ch:li for li,ch in int2char.items() }
print(char2int)
encoded = np.array([char2int[ch] for ch in text])
encoded[:100]

{'W': 0, ' ': 1, 'v': 2, 'g': 3, 'P': 4, '2': 5, 'y': 6, 'c': 7, 'r': 8, 'C': 9, '7': 10, 's': 11, '9': 12, 'i': 13, '(': 14, 'X': 15, '%': 16, 'K': 17, "'": 18, 'A': 19, 'k': 20, 'l': 21, 'U': 22, '4': 23, 'R': 24, 't': 25, 'F': 26, 'o': 27, 'S': 28, '_': 29, 'H': 30, '3': 31, 'n': 32, 'Y': 33, '/': 34, '@': 35, 'N': 36, 'm': 37, 'e': 38, 'O': 39, '1': 40, 'j': 41, 'Z': 42, '8': 43, '*': 44, '5': 45, '$': 46, 'Q': 47, '`': 48, '.': 49, 'G': 50, 'u': 51, 'a': 52, ';': 53, ')': 54, '!': 55, '-': 56, '&': 57, 'x': 58, 'z': 59, 'b': 60, 'T': 61, '\n': 62, 'q': 63, '"': 64, 'I': 65, ':': 66, '6': 67, '0': 68, 'd': 69, 'h': 70, 'p': 71, 'f': 72, ',': 73, 'D': 74, 'B': 75, 'L': 76, 'M': 77, 'J': 78, 'E': 79, '?': 80, 'w': 81, 'V': 82}


array([ 9, 70, 52, 71, 25, 38,  8,  1, 40, 62, 62, 62, 30, 52, 71, 71,  6,
        1, 72, 52, 37, 13, 21, 13, 38, 11,  1, 52,  8, 38,  1, 52, 21, 21,
        1, 52, 21, 13, 20, 38, 53,  1, 38,  2, 38,  8,  6,  1, 51, 32, 70,
       52, 71, 71,  6,  1, 72, 52, 37, 13, 21,  6,  1, 13, 11,  1, 51, 32,
       70, 52, 71, 71,  6,  1, 13, 32,  1, 13, 25, 11,  1, 27, 81, 32, 62,
       81, 52,  6, 49, 62, 62, 79,  2, 38,  8,  6, 25, 70, 13, 32])

In [0]:
def one_hot_encode(arr,n_labels):
  one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1
  one_hot = one_hot.reshape((*arr.shape, n_labels))
    
  return one_hot
def one_hot_encod(arr, n_labels):
  one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
  one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
  one_hot = one_hot.reshape((*arr.shape, n_labels))
  return one_hot  


test_seq = np.array([[3, 5, 1]])
print(test_seq.size)
one_hot = one_hot_encod(test_seq, 8)

print(one_hot)

3
[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


batches = get_batches(encoded, 8, 50)
x, y = next(batches) 
print('x\n', x)
print('\ny\n', y) 
print(len(encoded))      

x
 [[ 9 70 52 71 25 38  8  1 40 62 62 62 30 52 71 71  6  1 72 52 37 13 21 13
  38 11  1 52  8 38  1 52 21 21  1 52 21 13 20 38 53  1 38  2 38  8  6  1
  51 32]
 [11 27 32  1 25 70 52 25  1 52 25 25  8 52  7 25 38 69  1 70 38  8  1 52
  25 25 38 32 25 13 27 32  1 81 52 11  1 70 38  8  1 70 51 11 60 52 32 69
  49  1]
 [38 32 69  1 27  8  1 52  1 72 27 38 73  1 70 38  1 52  2 27 13 69 38 69
   1 70 13 11  1 72 52 25 70 38  8 49  1 30 38 62 21 27 27 20 38 69  1  8
  27 51]
 [11  1 25 70 38  1  7 70 13 38 72  1 25 70 27 51  3 70  1 70 13 69 69 38
  32 62 13 32 25 38  8 38 11 25  1 27 72  1 70 13 11  1 21 13 72 38 73  1
  27 72]
 [ 1 11 52 81  1 70 38  8  1 25 38 52  8 56 11 25 52 13 32 38 69 73  1 71
  13 25 13 72 51 21 73  1 11 81 38 38 25  1 72 52  7 38 73 62 37 13 11 38
   8 52]
 [ 7 51 11 11 13 27 32  1 52 32 69  1 52 32 52 21  6 11 13 11 73  1 81 52
  11  1 13 32  1 71  8 13 32  7 13 71 21 38  1 69 13 11 52  3  8 38 38 52
  60 21]
 [ 1 19 32 32 52  1 70 52 69  1 11 52 13 69  1 25 70 52

In [0]:
class RNN(nn.Module):
  def __init__ (self,token,hidden = 256,n_layers=2,drop_pb =0.5 ,lr=0.001):
    super(). __init__ ()
    self.hidden=hidden
    self.n_layers =n_layers
    self.drop_pb =drop_pb
    self.lr=lr

    self.char=token
    self.int2char = dict(enumerate(self.char))
    self.char2int = {ch: ii for ii, ch in self.int2char.items()}


    self.lstm=nn.LSTM(len(self.char), hidden,n_layers,batch_first=True,dropout=drop_pb)
    self.dropout=nn.Dropout(p=drop_pb)
    self.fc = nn.Linear(hidden,len(self.char))


  def forward(self,x,hidden):


    output,hidden=self.lstm(x,hidden)
    out=self.dropout(output)
    out=out.contiguous().view(-1,self.hidden)
    out = self.fc(out)

    return out,hidden


  def init_hidden(self,batch_size):
    change= next(self.parameters()).data
    hidden = (change.new(self.n_layers, batch_size, self.hidden).zero_(),
      change.new(self.n_layers, batch_size, self.hidden).zero_())
    return hidden


    







# **TRAINING THE** **MODEL**

In [0]:
def train(model,data,batch_size,epochs,lr,seq_length,val_frac,print_every,clip=5):

  model.train()
  count=0
  n_char=len(model.char)
  optimizer=torch.optim.Adam(model.parameters(),lr=lr)
  criterian=nn.CrossEntropyLoss()
  val_indx=int(len(data)*(1-val_frac))
  data,val_data=data[:val_indx],data[val_indx:]



  for i in range(epochs):
    h = model.init_hidden(batch_size)
    


    for x,y in get_batches(data,batch_size,seq_length):
      count=count+1
      
      
      x= one_hot_encod(x,n_char)
      inputs,targets = torch.from_numpy(x),torch.from_numpy(y)
      h=tuple([each.data for each in h])
      #print(h)

      model.zero_grad()


      outputs,h = model(inputs,h)
      
      
      loss=criterian(outputs, targets.view(batch_size*seq_length).long())
      loss.backward()
      nn.utils.clip_grad_norm_(model.parameters(), clip)
      optimizer.step()
      if count % print_every == 0:
        print("Epoch: {}/{}...".format(i+1, epochs),
                      "Step: {}...".format(count),
                      "Loss: {:.4f}...".format(loss.item()))
                      #Val Loss: {:.4f}".format(np.mean(val_losses)))


      '''if count % print_every == 0:
        val_h = model.init_hidden(batch_size)
        val_losses = []
        model.eval()
        for x, y in get_batches(val_data, batch_size, seq_length):

                    
          x = one_hot_encod(x, n_char)
          x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
               
          val_h = tuple([each.data for each in val_h])
                    
          inputs, targets = x, y
          
          output, val_h = model(inputs, val_h)
          #print(targets.view(batch_size*seq_length))
          
          
          val_loss = criterian(output, targets.view(batch_size*seq_length).long())
                
          val_losses.append(val_loss.item())
                
          model.train() # reset to train mode after iterationg through validation data
      if count % print_every == 0:
        print("Epoch: {}/{}...".format(i+1, epochs),
                      "Step: {}...".format(count),
                      "Loss: {:.4f}...".format(loss.item()))
                      #Val Loss: {:.4f}".format(np.mean(val_losses)))'''

      

      

          







In [0]:
n_hidden=512
n_layers=2

model = RNN(char, n_hidden, n_layers)
print(model)

RNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [0]:
batch_size = 128
seq_length = 100
n_epochs = 2
# train the model
train(model, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10,val_frac=0.1)

Epoch: 1/2... Step: 10... Loss: 3.2618...
Epoch: 1/2... Step: 20... Loss: 3.1546...
Epoch: 1/2... Step: 30... Loss: 3.1419...
Epoch: 1/2... Step: 40... Loss: 3.1118...
Epoch: 1/2... Step: 50... Loss: 3.1422...
Epoch: 1/2... Step: 60... Loss: 3.1143...
Epoch: 1/2... Step: 70... Loss: 3.1029...
Epoch: 1/2... Step: 80... Loss: 3.1030...
Epoch: 1/2... Step: 90... Loss: 3.0843...
Epoch: 1/2... Step: 100... Loss: 3.0041...
Epoch: 1/2... Step: 110... Loss: 2.9673...
Epoch: 1/2... Step: 120... Loss: 2.8302...
Epoch: 1/2... Step: 130... Loss: 2.7813...
Epoch: 2/2... Step: 140... Loss: 2.6935...
Epoch: 2/2... Step: 150... Loss: 2.6126...
Epoch: 2/2... Step: 160... Loss: 2.5398...
Epoch: 2/2... Step: 170... Loss: 2.4715...
Epoch: 2/2... Step: 180... Loss: 2.4388...
Epoch: 2/2... Step: 190... Loss: 2.3965...
Epoch: 2/2... Step: 200... Loss: 2.3800...
Epoch: 2/2... Step: 210... Loss: 2.3533...
Epoch: 2/2... Step: 220... Loss: 2.3090...
Epoch: 2/2... Step: 230... Loss: 2.3007...
Epoch: 2/2... Step: 

In [0]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': model.hidden,
              'n_layers': model.n_layers,
              'state_dict': model.state_dict(),
              'tokens': model.char}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def Learning (data,batch_size,seq_length):
  n_char=len(model.char)
  count=0
  for x, y in get_batches(data, batch_size, seq_length):
    x = one_hot_encod(x, n_char)
    x, y = torch.from_numpy(x), torch.from_numpy(y)
    count=count+1                
               
    #val_h = tuple([each.data for each in val_h])
                    
    inputs, targets = x, y
    #from torch import tensor
    print(x.size())
    print(y.size())
    #print(len(y))
    p=targets.flatten()
    print(p.size())
    print(count)      
    #output, val_h = model(inputs, val_h)
    #print(len(targets.view(batch_size*seq_length)))

In [0]:
Learning (data=encoded,batch_size=128,seq_length=100)

torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
1
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
2
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
3
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
4
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
5
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
6
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
7
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
8
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
9
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
10
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
11
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
12
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size([12800])
13
torch.Size([128, 100, 83])
torch.Size([128, 100])
torch.Size

In [0]:
z = 155*128*100
print(z)
print(len(encoded))

1984000
1985223
